# S2 explorations

* [Google S2 geometry blog](https://blog.christianperone.com/2015/08/googles-s2-geometry-on-the-sphere-cells-and-hilbert-curve/)
* [s2 site](https://s2geometry.io/)
* [GeoParquet index issue](https://github.com/opengeospatial/geoparquet/issues/13)
* [Boundiing box tool](https://boundingbox.klokantech.com/)
* http://s2.sidewalklabs.com/
* https://npm.runkit.com/s2-geometry
* https://github.com/MzHub/osmcoverer
* [s2 space filling](https://medium.com/sidewalk-talk/s2-cells-and-space-filling-curves-keys-to-building-better-digital-map-tools-for-cities-a312aa5e2f59)
* [understanding s2 at stackexchange](https://gis.stackexchange.com/questions/236508/understanding-the-s2-library-geometry-on-the-sphere-cells-and-hilbert-curve-f)
* https://github.com/google/s2geometry
* https://github.com/sidewalklabs/s2sphere

[Example s2 cell example](http://s2.sidewalklabs.com/regioncoverer/?center=39.541983%2C-82.031665&zoom=14&cells=884805515%2C884805517%2C884805519%2C884805523%2C884805525%2C88480552c%2C884805534%2C88480553c%2C88480555%2C884805564%2C88480556c%2C88480fe1%2C88480fe3%2C88480fe44%2C88480fe4c%2C88480fe6c%2C88480fe73%2C88480ff8c%2C88480ff94%2C88480ffb4%2C88480ffbc%2C88480ffd%2C88480fff%2C88481004%2C8848100c%2C88481011%2C88481013%2C884810144%2C88481014c%2C88481016c%2C884810174%2C884810194%2C88481019c%2C8848101b%2C8848101d%2C8848101f%2C884810614%2C88481061c%2C88481063%2C88481065%2C884810664%2C88481066c%2C88481068c%2C884810694%2C8848106b4%2C8848106bc%2C8848106d%2C8848106f%2C88481074%2C8848107c%2C88481084%2C88481089%2C8848108f%2C88481a62c%2C88481a634%2C88481a64c%2C88481a654%2C88481a7c%2C88481a84%2C88481a884%2C88481a88c%2C88481a8f4%2C88481a8fc%2C88481a904%2C88481a90c%2C88481a9c%2C88481aa4%2C88481aac)

https://s2sphere.readthedocs.io/en/latest/api.html

In [1]:
import s2sphere as s2
from s2sphere import CellId, LatLng, Cell

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import folium
import geopandas as gpd
from shapely.geometry import Polygon

In [2]:
# Simple cell id generation
def s3rc(lmin, lmax, cmax, p1lat, p1lng, p2lat, p2lng):
    r = s2.RegionCoverer()
    r.min_level=lmin
    r.max_level=lmax
    r.max_cells=cmax
    p1 = s2.LatLng.from_degrees(p1lat, p1lng)
    p2 = s2.LatLng.from_degrees(p2lat, p2lng)
    cell_ids = r.get_covering(s2.LatLngRect.from_point_pair(p1, p2))
    return(cell_ids)

In [3]:
def get_corners(s2CellId_str_in, level):
    s2CellId_str = s2CellId_str_in.rstrip("0")
    c1 = Cell(CellId(int(s2CellId_str,16)<<(60 - 2*level)))
    c0 = LatLng.from_point(c1.get_center())  # center lat/lon of s2 cell
    v0 = LatLng.from_point(c1.get_vertex(0)) # lat/lon of upper/left corner
    v1 = LatLng.from_point(c1.get_vertex(1)) # lat/lon of lower/left corner
    v2 = LatLng.from_point(c1.get_vertex(2)) # lat/lon of lower/right corner
    v3 = LatLng.from_point(c1.get_vertex(3)) # lat/lon of upper/right corner
    lat_point_list = [v0.lat().degrees,v1.lat().degrees,v2.lat().degrees,v3.lat().degrees]
    lon_point_list = [v0.lng().degrees,v1.lng().degrees,v2.lng().degrees,v3.lng().degrees]
    return lat_point_list, lon_point_list


In [4]:
def return_polygons(covering):
    polys = []
    for cellid in covering:
        new_cell = Cell(cellid)
        c = str(new_cell.id()).split()
        x, y = get_corners(c[1], new_cell.level())

        # print("{0}-----------------{1}".format(c[1], cellid.level()))

        polygon_geom = Polygon(zip(y, x))
        crs = {'init': 'epsg:4326'}
        polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])
        polys.append(polygon)

    return polys

In [5]:
lmin = 12
lmax = 12
cmax = 100
p1lat = 40.78306025613585
p1lng = -73.97124880430383
p2lat = 40.78306025613585
p2lng = -73.97124880430383

covering = s3rc(lmin, lmax, cmax, p1lat, p1lng, p2lat, p2lng)
print("-------Print Covering------")
print(covering)


polygons = return_polygons(covering)
print("\n------Print polygons-------")
print(polygons)


x, y = get_corners("89c2589", 12) # S2 Level 12 cell id in Central Park, New York
print("\n------Print corners of grids-------")
print(x)
print(y)


-------Print Covering------
[CellId: 89c2589000000000]

------Print polygons-------
[                                            geometry
0  POLYGON ((-73.98300 40.79460, -73.98300 40.774...]

------Print corners of grids-------
[40.79460381328247, 40.77485404943761, 40.77151380930433, 40.79126323170246]
[-73.9830029344863, -73.98300293448631, -73.95949395655923, -73.95949395655923]


In [6]:


m = folium.Map([40.78306025613585, -73.97124880430383], zoom_start=13, tiles='cartodbpositron')

# lat_point_list = [40.79460381328247,40.77485404943761, 40.77151380930433,40.79126323170246] # [50.854457, 52.518172, 50.072651, 48.853033, 50.854457]
# lon_point_list = [-73.9830029344863,-73.98300293448631,-73.95949395655923,-73.95949395655923] #[4.377184, 13.407759, 14.435935, 2.349553, 4.377184]

# 89c2589

x, y = get_corners("89c2589", 12)
print(x)
print(y)

polygon_geom = Polygon(zip(y, x))
print(polygon_geom)
crs = {'init': 'epsg:4326'}
polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])
folium.GeoJson(polygon).add_to(m)
folium.LatLngPopup().add_to(m)

# show map
m

[40.79460381328247, 40.77485404943761, 40.77151380930433, 40.79126323170246]
[-73.9830029344863, -73.98300293448631, -73.95949395655923, -73.95949395655923]
POLYGON ((-73.9830029344863 40.79460381328247, -73.98300293448631 40.77485404943761, -73.95949395655923 40.77151380930433, -73.95949395655923 40.79126323170246, -73.9830029344863 40.79460381328247))


In [11]:
import folium
import geopandas as gpd
from shapely.geometry import Polygon

m = folium.Map([40.78306025613585, -73.97124880430383], zoom_start=13, tiles='cartodbpositron')

lmin = 12
lmax = 12
cmax = 100
p1lat = 40.78306025613585
p1lng = -73.97124880430383
p2lat = 40.78306025613585
p2lng = -73.97124880430383

covering = s3rc(lmin, lmax, cmax, p1lat, p1lng, p2lat, p2lng)

polygons = return_polygons(covering)
# print(polygons)

for p in polygons:
    print(p)
    folium.GeoJson(p).add_to(m)

# show map
m

                                            geometry
0  POLYGON ((-73.98300 40.79460, -73.98300 40.774...


In [12]:
import folium
import geopandas as gpd
from shapely.geometry import Polygon


# Upper left: Latitude: 39.5804   Longitude: -84.0379
# Lower right: Latitude: 39.4924  Longitude: -83.9218
lmin = 14
lmax = 14
cmax = 500
p1lat = 39.5804
p1lng = -84.0379
p2lat = 39.4924
p2lng = -83.9218

covering = s3rc(lmin, lmax, cmax, p1lat, p1lng, p2lat, p2lng) 

m = folium.Map([39.5293, -83.9907], zoom_start=12, tiles='cartodbpositron')

polygons = return_polygons(covering)
# print(polygons)

print(len(polygons))
      
for p in polygons:
    folium.GeoJson(p).add_to(m)
    
folium.Rectangle(bounds = [(p1lat, p1lng), (p2lat, p2lng)], color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.2).add_to(m)


# print(gs)
# folium.GeoJson(gs, name="geojson").add_to(m)
folium.LatLngPopup().add_to(m)

m


417


## KnowWhere Graph

[https://stko-kwg.geog.ucsb.edu/browse/#kwgr:s2.level13.9836861334304587776](https://stko-kwg.geog.ucsb.edu/browse/#kwgr:s2.level13.9836861334304587776)

In [13]:
lmin = 13
lmax = 13
cmax = 100
p1lat = 40.78306025613585
p1lng = -73.97124880430383
p2lat = 40.78306025613585
p2lng = -73.97124880430383

covering = s3rc(lmin, lmax, cmax, p1lat, p1lng, p2lat, p2lng)
print("-------Print Covering------")
print(covering)

c = str(covering[0].id()).split()
print(c)

-------Print Covering------
[CellId: 89c2588400000000, CellId: 89c2588c00000000, CellId: 89c2589400000000, CellId: 89c2589c00000000]
['9926593852636921856']


NYC example can be seen at [KWG at 9926593852636921856](https://stko-kwg.geog.ucsb.edu/browse/#kwgr:s2.level13.9926593852636921856)

In [14]:
# for an Ohio example

lmin = 13
lmax = 13
cmax = 500
p1lat = 39.5804
p1lng = -84.0379
p2lat = 39.4924
p2lng = -83.9218

covering = s3rc(lmin, lmax, cmax, p1lat, p1lng, p2lat, p2lng)
# print("-------Print Covering------")
# print(covering)

c = str(covering[0].id()).split()
print(c)

['9818008867416571904']


The info from [KWG for 9818008867416571904](https://stko-kwg.geog.ucsb.edu/browse/#kwgr:s2.level13.9818008867416571904)